In [92]:
import numpy as np 
import pandas as pd 
from sklearn.naive_bayes import GaussianNB

In [93]:
df  = pd.read_csv('titanic2.csv')
df.head(3)

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1


In [94]:
training_df  = df.drop(['PassengerId','Name',"SibSp",'Parch','Ticket','Cabin','Embarked'],axis='columns')
training_df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [95]:
dummy = pd.get_dummies(training_df.Sex)
dummy.head()

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True


**male in also drop to get rid of dummy variable trap theory**

In [96]:
input = training_df.drop(['Survived',"Sex"],axis='columns')
input = pd.concat([input,dummy],axis='columns')
input = input.drop('male',axis='columns')
input.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [97]:
input.columns[input.isna().any()]



Index(['Age'], dtype='object')

In [98]:
input.Age = input.Age.fillna(input.Age.mean())
input.head(10)

,Pclass,Age,Fare,female
0,3,22.000000,7.2500,False
1,1,38.000000,71.2833,True
2,3,26.000000,7.9250,True
3,1,35.000000,53.1000,True
4,3,35.000000,8.0500,False
5,3,29.699118,8.4583,False
6,1,54.000000,51.8625,False
7,3,2.000000,21.0750,False
8,3,27.000000,11.1333,True
9,2,14.000000,30.0708,True


In [99]:
output = training_df.Survived
output.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [100]:
output.isna().any()

False

In [101]:
from sklearn.model_selection import train_test_split


In [102]:
train_x ,test_x,train_y,test_y = train_test_split(input,output,train_size=0.8)


In [103]:
train_x.head()

,Pclass,Age,Fare,female
813,3,6.000000,31.2750,True
69,3,26.000000,8.6625,False
412,1,33.000000,90.0000,True
870,3,26.000000,7.8958,False
158,3,29.699118,8.6625,False


In [104]:
train_y.head()

813    0
69     0
412    1
870    0
158    0
Name: Survived, dtype: int64

In [105]:
test_x.head()

,Pclass,Age,Fare,female
339,1,45.0,35.5000,False
609,1,40.0,153.4625,True
423,3,28.0,14.4000,True
477,3,29.0,7.0458,False
93,3,26.0,20.5750,False


In [106]:
test_y.head()

339    0
609    1
423    0
477    0
93     0
Name: Survived, dtype: int64

In [107]:
model = GaussianNB()


In [108]:
model.fit(train_x,train_y)
model.score(test_x,test_y)

0.8324022346368715

In [109]:
model.predict_proba(train_x[:10])

array([[0.29632256, 0.70367744],
       [0.9595237 , 0.0404763 ],
       [0.01749466, 0.98250534],
       [0.95946953, 0.04053047],
       [0.96108875, 0.03891125],
       [0.91790379, 0.08209621],
       [0.40318672, 0.59681328],
       [0.95646404, 0.04353596],
       [0.91785905, 0.08214095],
       [0.95426627, 0.04573373]])

**using Cross_val_score**

In [110]:
from sklearn.model_selection import cross_val_score

In [111]:
cross_val_score(GaussianNB(),input,output)

array([0.70949721, 0.79775281, 0.78089888, 0.75842697, 0.79213483])

## Excersice 

In [112]:
import pandas as pd 

In [113]:
email = pd.read_csv('spam.csv')
email.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [114]:
email.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [115]:
email['spam']=email['Category'].apply(lambda x : 1 if x == 'spam' else 0)
email.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [116]:
from sklearn.model_selection import train_test_split

In [117]:
X_train,X_test,Y_train,Y_test = train_test_split(email.drop('spam',axis='columns'),email.spam,train_size=0.8)

In [118]:
len(X_train),len(X_test),len(Y_train),len(Y_test)

(4457, 1115, 4457, 1115)

In [119]:
X_train.head()

,Category,Message
1078,ham,"Yep, by the pretty sculpture"
3620,spam,8007 25p 4 Alfie Moon's Children in Need song ...
2894,ham,K I'll be sure to get up before noon and see w...
2385,ham,"Sorry, I'll call later"
4843,ham,"Awesome, that gonna be soon or later tonight?"


In [120]:
X_train = X_train.drop('Category',axis = 'columns')
X_train.head()

,Message
1078,"Yep, by the pretty sculpture"
3620,8007 25p 4 Alfie Moon's Children in Need song ...
2894,K I'll be sure to get up before noon and see w...
2385,"Sorry, I'll call later"
4843,"Awesome, that gonna be soon or later tonight?"


In [121]:
from sklearn.feature_extraction.text import CountVectorizer

In [122]:
v =CountVectorizer()

In [125]:
X_train_Count = v.fit_transform(X_train['Message'].values)
X_train_Count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [126]:
from sklearn.naive_bayes import MultinomialNB

In [129]:
X_test_count = v.transform(X_test['Message'].values)
X_test_count.toarray()[:2]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [128]:
model = MultinomialNB()
model.fit(X_train_Count,Y_train)


MultinomialNB()

In [130]:
model.score(X_test_count,Y_test)

0.9919282511210762

In [140]:
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!' 
]
#1. ham 
#2. spam 


In [141]:
email_count = v.transform(emails)
model.predict(email_count)

array([0, 1], dtype=int64)

## Sklearn Pipeline 

In [142]:
from sklearn.pipeline import Pipeline

In [146]:
clf = Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [148]:
clf.fit(X_train['Message'],Y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [149]:
clf.score(X_test['Message'],Y_test)

0.9919282511210762

In [151]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Assuming you have initialized X_train and Y_train

# Create the pipeline
clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

# Fit the pipeline on the training data
clf.fit(X_train['Message'], Y_train)

# Optionally, predict on a few examples
# Assuming you have initialized X_test
predictions = clf.predict(X_test['Message'])

# Print some information for debugging
print("Pipeline components:", clf.named_steps.keys())
print("Vocabulary size:", len(clf.named_steps['vectorizer'].get_feature_names_out()))
print("Number of features in transformed X_train:", clf.named_steps['vectorizer'].transform(X_train['Message']).shape[1])
print("Number of samples in X_train:", clf.named_steps['vectorizer'].transform(X_train['Message']).shape[0])

# If needed, print the predictions
print("Predictions:", predictions)


Pipeline components: dict_keys(['vectorizer', 'nb'])
Vocabulary size: 7702
Number of features in transformed X_train: 7702
Number of samples in X_train: 4457
Predictions: [0 0 0 ... 0 0 0]
